In [1]:
import numpy as np
import pandas as pd
import json
from utils import *
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, GRU, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

In [44]:
# truth_df = pd.DataFrame(columns=['id','truthMedian','truthClass','truthMean'])
# with open('data/truth.jsonl') as data:
#     for labelobj in data:
#         truth = json.loads(labelobj)
#         truthlabel = {'id': truth['id'], 'truthMedian': truth['truthMedian'], 'truthClass': truth['truthClass'], 'truthMean': truth['truthMean']}
#         truth_df = truth_df.append(truthlabel, ignore_index = True)
# truth_df.head()  

In [45]:
# instances_df = pd.DataFrame(columns=['id','postText'])
# with open('data/instances.jsonl') as data:
# 	for instanceobj in data:
# 		instance = json.loads(instanceobj)
# 		instancerow = {'id': instance['id'], 'postText': instance['postText']}
# 		instances_df = instances_df.append(instancerow, ignore_index=True)
# instances_df.head()

In [46]:
# dataset = instances_df.join(truth_df.set_index('id'), on='id')
# dataset = dataset.drop(labels='id',axis=1)
# for i in range(len(dataset)):
#     dataset['postText'].values[i] = dataset['postText'].values[i][0]
# dataset['postText'].dropna(inplace=True)
# dataset.head()

In [47]:
# # Convert the 'no-clickbait' or 'clickbait' to binary indicator
# dataset['truthClass'] = dataset['truthClass'].apply(classToBinary)
# # Convert floating number in 'truthMedian' column to integer
# dataset['truthMedian'] = dataset['truthMedian'].apply(medianToInteger)
# # Remove all punctuations and clear the text
# dataset['postText'] = dataset['postText'].apply(cleanText)
# dataset.head()

In [48]:
# # split the dataset to training and testing set
# train, test = train_test_split(dataset, test_size=0.2)
# X_train, Y_train = np.array(train["postText"].tolist()), np.array(train["truthClass"].tolist())
# X_test, Y_test = np.array(test["postText"].tolist()), np.array(test["truthClass"].tolist())
# print(X_train.shape)
# print(X_test.shape)

In [24]:
train_df = pd.DataFrame(columns=['id','postText','truthMedian','truthClass','truthMean'])
with open('data/train.json') as json_data:
    data = json.load(json_data)
    for instance in data:
        train_instance = {'id': instance['text_id'], 'postText': instance['post_text'], 'truthMedian': instance['truth_median'], 'truthClass': instance['click_bait'], 'truthMean': instance['truth_mean']}
        train_df = train_df.append(train_instance, ignore_index = True)
train_df.head()

,id,postText,truthMedian,truthClass,truthMean
0,849630396191055872,"Flying mattress strikes biker at 50 mph, ends ...",0.000000,0,0.200000
1,805003400136036352,The Grand Tour: Jeremy Clarkson is really taun...,0.000000,0,0.200000
2,811134324628226048,Was this murderous attack on a Russian diploma...,0.666667,1,0.533333
3,807681539580502016,"PartyNextDoor brings out Drake, G-Eazy and mor...",0.000000,0,0.266667
4,807465142032232450,A far-right Dutch lawmaker who wants to Make t...,0.000000,0,0.200000


In [25]:
test_df = pd.DataFrame(columns=['id','postText','truthMedian','truthClass','truthMean'])
with open('data/test.json') as json_data:
    data = json.load(json_data)
    for instance in data:
        test_instance = {'id': instance['text_id'], 'postText': instance['post_text'], 'truthMedian': instance['truth_median'], 'truthClass': instance['click_bait'], 'truthMean': instance['truth_mean']}
        test_df = test_df.append(test_instance, ignore_index = True)
test_df.head()

,id,postText,truthMedian,truthClass,truthMean
0,841383755944189952,"John Lewis blasts GOP rep's ""bigoted and racis...",0.000000,0,0.200000
1,852354010820620288,The NBA playoff bracket is now set,0.000000,0,0.200000
2,838728651545903104,Migrants smash up German asylum centre and sev...,0.000000,0,0.133333
3,805465449739718656,"America's most livable states, ranked",0.666667,1,0.666667
4,841713776382627840,WIP or RIP?,1.000000,1,0.933333


In [26]:
# Convert floating number in 'truthMedian' column to integer
train_df['truthMedian'] = train_df['truthMedian'].apply(medianToInteger)
# Remove all punctuations and clear the text
train_df['postText'] = train_df['postText'].apply(cleanText)
train_df.head()

,id,postText,truthMedian,truthClass,truthMean
0,849630396191055872,flying mattress strikes biker at 50 mph ends u...,0,0,0.200000
1,805003400136036352,the grand tour jeremy clarkson is really taunt...,0,0,0.200000
2,811134324628226048,was this murderous attack on a russian diploma...,2,1,0.533333
3,807681539580502016,partynextdoor brings out drake geazy and more ...,0,0,0.266667
4,807465142032232450,a farright dutch lawmaker who wants to make th...,0,0,0.200000


In [27]:
# Convert floating number in 'truthMedian' column to integer
test_df['truthMedian'] = test_df['truthMedian'].apply(medianToInteger)
# Remove all punctuations and clear the text
test_df['postText'] = test_df['postText'].apply(cleanText)
test_df.head()

,id,postText,truthMedian,truthClass,truthMean
0,841383755944189952,john lewis blasts gop reps bigoted and racist ...,0,0,0.200000
1,852354010820620288,the nba playoff bracket is now set,0,0,0.200000
2,838728651545903104,migrants smash up german asylum centre and sev...,0,0,0.133333
3,805465449739718656,americas most livable states ranked,2,1,0.666667
4,841713776382627840,wip or rip,3,1,0.933333


In [28]:
X_train, Y_train = np.array(train_df["postText"].tolist()), np.array(train_df["truthClass"].tolist())
X_test, Y_test = np.array(test_df["postText"].tolist()), np.array(test_df["truthClass"].tolist())
print(X_train.shape)
print(X_test.shape)

(15567,)
(3892,)


In [19]:
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
import matplotlib.pyplot as plt


C:\Users\yuanb\AppData\Roaming\Python\Python37\site-packages\tensorflow_addons\utils\ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.0-rc0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [14]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'

map_name_to_handle = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
}

map_model_to_preprocess = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1'
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1


In [20]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [21]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [29]:
def Bert_Model():
    text_input = Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = Dropout(0.1)(net)
    net = Dense(1, activation=None, name='classifier')(net)
    return Model(text_input, net)

In [30]:
bert_model = Bert_Model()
bert_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_word_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
BERT_encoder (KerasLayer)       {'sequence_output':  28763649    preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
____________________________________________________________________________________________

In [31]:
bert_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# classifier_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
#history = bert_model.fit(X_train, Y_train, validation_data=val_ds, epochs=epochs)
bert_model.fit(X_train, Y_train, epochs = 5, batch_size = 64, shuffle=True)

Epoch 1/5


InvalidArgumentError:  Trying to access resource using the wrong type. Expected class tensorflow::lookup::LookupInterface got class tensorflow::lookup::LookupInterface
	 [[{{node model_2/preprocessing/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/bert_tokenizer/StatefulPartitionedCall/WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets}}]] [Op:__inference_train_function_303597]

Function call stack:
train_function


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,roc_auc_score, mean_squared_error,classification_report
loss, accuracy = classifier_model.evaluate(X_test, Y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')
Y_predict = classifier_model.predict(X_test)
Y_predict_binary = [1 if i >= 0.5 else 0 for [i] in Y_predict]
print(classification_report(Y_test, Y_predict_binary))